<a href="https://colab.research.google.com/github/ncai25/reddit-depression/blob/main/Reddit_Depression_Stencil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reddit Depression Final Project
Link to the paper: https://dl.acm.org/doi/pdf/10.1145/3578503.3583621

## Preprocessing

In [3]:
!pip install --upgrade pandas
!pip install gensim

import pickle
import os
from os.path import exists
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_validate, cross_val_score, KFold
from sklearn.ensemble import RandomForestClassifier


from gensim.corpora import Dictionary
# from gensim.models import LdaMulticore
from gensim.models import LdaModel

from google.colab import drive
drive.mount('/content/drive')

FILEPATH = 'drive/MyDrive/Colab Notebooks/final_project/student.pkl'
FOLDER = 'drive/MyDrive/Colab Notebooks/final_project'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Data Loading

In [11]:
def load():
    """Load pickles"""
    if exists(FILEPATH):
        with open(FILEPATH, 'rb') as f:
            data = pd.read_pickle(f)
        return data
    else:
        print(f"File {FILEPATH} does not exist.")
        return None

df = load()

In [12]:
# List of depression subreddits in the paper
depression_subreddits = ["Anger",
    "anhedonia", "DeadBedrooms",
    "Anxiety", "AnxietyDepression", "HealthAnxiety", "PanicAttack",
    "DecisionMaking", "shouldi",
    "bingeeating", "BingeEatingDisorder", "EatingDisorders", "eating_disorders", "EDAnonymous",
    "chronicfatigue", "Fatigue",
    "ForeverAlone", "lonely",
    "cry", "grief", "sad", "Sadness",
    "AvPD", "SelfHate", "selfhelp", "socialanxiety", "whatsbotheringyou",
    "insomnia", "sleep",
    "cfs", "ChronicPain", "Constipation", "EssentialTremor", "headaches", "ibs", "tinnitus",
    "AdultSelfHarm", "selfharm", "SuicideWatch",
    "Guilt", "Pessimism", "selfhelp", "whatsbotheringyou"
]

In [13]:
def dataset_generation(df, depression_subreddits):
    """Build control and symptom datasets
    For the control dataset, only posts were at least 180 days earlier than their (earliest) post in a mental health subreddit.”
    """
    # a sympton dataset consists of all the posts from depression subreddits
    df_depression = df[df['subreddit'].isin(depression_subreddits)]

    # a control dataset posts from non-depression subreddits of the same authors
    authors = df_depression['author'].unique()
    df_control = df[(df['author'].isin(authors)) & (~df['subreddit'].isin(depression_subreddits))]

    # find the earliest depression posts for each author
    grouped_author = df_depression.groupby('author')['created_utc'].min()
    author_to_earliest = grouped_author.to_dict()
    time_delta = 180 * 24 * 60 * 60

    # filter the control posts so that they occur at least 180 days before that timestamp
    df_control = df_control.copy()
    df_control['earliest'] = df_control['author'].map(author_to_earliest)
    df_control['delta'] = df_control['created_utc'] - df_control['earliest']
    df_control = df_control[df_control['delta'] < -time_delta]

    return df_depression, df_control

df_depression, df_control = dataset_generation(df, depression_subreddits)

In [14]:
# Tests, reference (depression_df: 95916; control_df: 4369)
print(len(df_depression))
print(len(df_control))

94514
4369


In [15]:
# Tokenization + Stopword Removal

!pip install happiestfuntokenizing
from happiestfuntokenizing.happiestfuntokenizing import Tokenizer
import string
from collections import Counter

def tokenize(text: str):
    """Tokenize a single text string into a list of tokens"""
    tokenizer = Tokenizer()
    tokens = tokenizer.tokenize(text)
    punctuation = set(string.punctuation)
    tokens = [token for token in tokens if token not in punctuation]
    return tokens

# def lower_and_remove_punctuation(text: str):
#     """Lowercase text and remove punctuation for text column for Roberta"""
#     text = text.lower()
#     punctuation = set(string.punctuation)
#     text = ''.join([char for char in text if char not in punctuation])
#     return text

def find_stopwords(control_dataset):
    """Find top 100 words from the control dataset to use as stop words."""

    all_tokens = []
    for tokens in (control_dataset['tokens'].tolist()):
        all_tokens.extend(tokens)

    freq = Counter(all_tokens)
    top_100_words = [word for word, _ in freq.most_common(100)]

    return top_100_words

  Preparing metadata (setup.py) ... done
  Created wheel for happiestfuntokenizing: filename=happiestfuntokenizing-0.0.7-py3-none-any.whl size=6711 sha256=c93a5d1b2fbdf24f41819ac99305490ab05836e60d74387874a83d36116fd8e5
  Stored in directory: /root/.cache/pip/wheels/bf/c9/4d/310f0c60855eb7b428558f29d93cf464dbb64c1b8628753395
Successfully built happiestfuntokenizing


In [16]:
# Load preprocessed data from Google Drive
with open(f'{FOLDER}/pre_depression.pkl', 'rb') as f:
    pre_depression = pickle.load(f)

with open(f'{FOLDER}/pre_control.pkl', 'rb') as f:
    pre_control = pickle.load(f)

In [17]:
# Testing if preprocessed data is loaded

print(pre_depression['tokens'].head())
print()
print(pre_control['tokens'].head())
print(pre_control['text'].head())

20    [trying, hi, sorry, writing, bad, headache, di...
39    [friend, blanking, feels, months, complete, sh...
67    [study, hall, social, anxiety, bruh, study, ha...
72                 [positive, thoughts, happy, publish]
79    [starting, blowup, mattress, today, very, “, d...
Name: tokens, dtype: object

315                         [man, love, bandicoot, crash]
651     [pc, 700-750, budget, gaming, high, ultra, set...
730     [price, gpus, down, bitcoin, mining, trying, s...
1354    [our, service, available, area, hey, totally, ...
1598                                                [wow]
Name: tokens, dtype: object
315              Man, I do love me some Bandicoot crash. 
651     How good is this PC for my 700-750$ budget? Wa...
730     When is the price of gpus going down? I know t...
1354    Our service is not available in your area. Hey...
1598                                                 Wow 
Name: text, dtype: object


### Preprocessing (only need to run once)

In [ ]:
def preprocess(depression_data, control_data):
    """Preprocess the depression and control datasets by tokenizing and removing stopwords"""
    control_data = control_data.copy()
    depression_data = depression_data.copy()

    # control_data['text'] = control_data['text'].apply(lower_and_remove_punctuation)
    # depression_data['text'] = depression_data['text'].apply(lower_and_remove_punctuation)

    # tokenize
    control_data['tokens'] = control_data['text'].apply(tokenize)
    depression_data['tokens'] = depression_data['text'].apply(tokenize)

    # stop word removal
    stopwords = find_stopwords(control_data)
    # print(stopwords)
    depression_data['tokens'] = depression_data['tokens'].apply(lambda tokens: [t for t in tokens if t not in stopwords])
    control_data['tokens'] = control_data['tokens'].apply(lambda tokens: [t for t in tokens if t not in stopwords])
    return depression_data, control_data

pre_depression, pre_control = preprocess(df_depression, df_control)

In [ ]:
# Save preprocessed data to Google Drive
with open(f'{FOLDER}/pre_depression.pkl', 'wb') as f:
    pickle.dump(pre_depression, f)

with open(f'{FOLDER}/pre_control.pkl', 'wb') as f:
    pickle.dump(pre_control, f)

## Reddit Topics with LDA

### Load LDA model

In [18]:
# Load LDA model
lda_model = LdaModel.load(f'{FOLDER}/lda_model.gensim')
dictionary = Dictionary.load(f'{FOLDER}/dictionary.gensim')

# Load dictionary and corpus
with open(f'{FOLDER}/combined_corpus.pkl', 'rb') as f:
    combined_corpus = pickle.load(f)
with open(f'{FOLDER}/control_corpus.pkl', 'rb') as f:
    control_corpus = pickle.load(f)

### LDA model (run once)

In [ ]:
all_tokens = pre_depression['tokens'].tolist() + pre_control['tokens'].tolist()
# LDA should be trained on the entire dataset(all posts)

# dictionary and corpus
dictionary = Dictionary(all_tokens)
combined_corpus = [dictionary.doc2bow(doc) for doc in all_tokens]
# create a BoW representation for each doc/post, term-document, doc/post represented as (word_id, count)

lda_model = LdaModel(
    combined_corpus,
    num_topics=200,
    id2word=dictionary,
    passes=1,
)

control_corpus = [dictionary.doc2bow(doc) for doc in pre_control['tokens']]

In [ ]:
print(dict(list(dictionary.token2id.items())[:10]))
print(combined_corpus[:2])

In [ ]:
# Save LDA model, dictionary, and corpus
lda_model.save(f'{FOLDER}/lda_model.gensim')
dictionary.save(f'{FOLDER}/dictionary.gensim')
with open(f'{FOLDER}/combined_corpus.pkl', 'wb') as f:
    pickle.dump(combined_corpus, f)

# Save control corpus for later use
with open(f'{FOLDER}/control_corpus.pkl', 'wb') as f:
    pickle.dump(control_corpus, f)

## (Distil)RoBERTa Embeddings

In [19]:
# TODO: My RoBERTa code!

# only 6 transformer blocks (run faster)
# extracting embeddings from the 5th layer for downstream classification
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilroberta-base")
model = AutoModelForMaskedLM.from_pretrained("distilbert/distilroberta-base", output_hidden_states=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert/distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm)

In [20]:
def get_distilroberta_embeddings(sentences, tokenizer, model, device):
    """
    Generate embeddings for a list of sentences using DistilRoBERTa.

    Returns:
    - embeddings: NumPy array of shape (num_sentences, embedding_dim).
    """
    all_embeddings = []
    batch_size = 16 # batches otherwise ram

    for i in range(0, len(sentences), batch_size):
        batch_sentences = sentences[i:i + batch_size]

        inputs = tokenizer(
            batch_sentences,
            return_tensors="pt", # pytorch tensor
            padding=True,
            truncation=True, # truncate sequences > max_length
            max_length=512 # max sequence length
        )

        inputs = {key: value.to(device) for key, value in inputs.items()}

        with torch.no_grad(): # not training the model, no need to track the gradients
            outputs = model(**inputs)
            hidden_states = outputs.hidden_states

        # 5th layer's output for embeddings
        selected_layer = hidden_states[5]  # shape: (batch_size, sequence_length, hidden_size)

        # sentence-level representation
        sentence_embeddings = selected_layer.mean(dim=1)  # shape: (batch_size, hidden_size)
        # each word/token has a single embedding representation.
        # To get the embedding for a post, average the embeddings of each token

        all_embeddings.append(sentence_embeddings.cpu())

    all_embeddings = torch.cat(all_embeddings, dim=0).numpy()

    return all_embeddings

## Main/Evaluation

### Set up

In [8]:
# Reference values for LDA and DistilRoBERTa
reference_values = {
    "Anger": {"LDA": 0.794, "DistilRoBERTa": 0.928},
    "Anhedonia": {"LDA": 0.906, "DistilRoBERTa": 0.956},
    "Anxiety": {"LDA": 0.837, "DistilRoBERTa": 0.952},
    "Disordered Eating": {"LDA": 0.905, "DistilRoBERTa": 0.952},
    "Loneliness": {"LDA": 0.806, "DistilRoBERTa": 0.907},
    "Sad Mood": {"LDA": 0.788, "DistilRoBERTa": 0.919},
    "Self-Loathing": {"LDA": 0.815, "DistilRoBERTa": 0.922},
    "Sleep Problems": {"LDA": 0.909, "DistilRoBERTa": 0.956},
    "Somatic Complaints": {"LDA": 0.880, "DistilRoBERTa": 0.925},
    "Worthlessness": {"LDA": 0.700, "DistilRoBERTa": 0.897}
}

In [21]:
# Prepare Symptom Dataset

# Not to evaluate on Fatigue, Concentration Deficit, Suicidal Thoughts
symptom_to_subreddits = {
    "Anger": ["Anger"],
    "Anhedonia": ["anhedonia", "DeadBedrooms"],
    "Anxiety": ["Anxiety", "AnxietyDepression", "HealthAnxiety", "PanicAttack"],
    # "Concentration Deficit": ["DecisionMaking", "shouldi"],
    "Disordered Eating": ["bingeeating", "BingeEatingDisorder", "EatingDisorders", "eating_disorders", "EDAnonymous"],
    # "Fatigue": ["chronicfatigue", "Fatigue"],
    "Loneliness": ["ForeverAlone", "lonely"],
    "Sad Mood": ["cry", "grief", "sad", "Sadness"],
    "Self-Loathing": ["AvPD", "SelfHate", "selfhelp", "socialanxiety", "whatsbotheringyou"],
    "Sleep Problems": ["insomnia", "sleep"],
    "Somatic Complaints": ["cfs", "ChronicPain", "Constipation", "EssentialTremor", "headaches", "ibs", "tinnitus"],
    # "Suicidal Thoughts": ["AdultSelfHarm", "selfharm", "SuicideWatch"],
    "Worthlessness": ["Guilt", "Pessimism", "selfhelp", "whatsbotheringyou"]
}

symptom_datasets = {}

for symptom, subreddits in symptom_to_subreddits.items():
    # Filter rows where the subreddit matches one of the subreddits for this symptom
    symptom_data = pre_depression[pre_depression['subreddit'].isin(subreddits)]
    symptom_datasets[symptom] = symptom_data

# anxiety_data = symptom_datasets["Anxiety"]
# print(len(anxiety_data))

### LDA evaluation

In [23]:
def get_topic_distributions(corpus, model, num_topics):
    """
    Represent each doc/post as a vector of probabilites over topics - features
    """
    distributions = []
    for doc in corpus:
        sparse_dist = model[doc]
        dense_dist = np.zeros(num_topics)
        for topic_id, prob in sparse_dist:
            dense_dist[topic_id] = prob
        distributions.append(dense_dist)
    return np.array(distributions)

In [24]:
def lda_rf_cross_validation():
    """
    It runs 5-fold cross validation with random forest to evaluate my LDA performance.
    """
    # TODO: Print my training and testing scores!
    for symptom, symptom_data in symptom_datasets.items():
        print(f"Running for symptom: {symptom}")
        # --- LDA Workflow ---
        symptom_corpus = [dictionary.doc2bow(doc) for doc in symptom_data['tokens']]

        num_topics = lda_model.num_topics
        X_symptom = get_topic_distributions(symptom_corpus, lda_model, num_topics)
        X_control = get_topic_distributions(control_corpus, lda_model, num_topics)

        X_lda = np.vstack([X_symptom, X_control]) # (num_total_posts, num_topics)
        # rows - posts, col - topic prob (features)
        y_lda = np.hstack([np.ones(X_symptom.shape[0]), np.zeros(X_control.shape[0])])
        # binary label; ground truth
        # positive class (1): posts exhibiting symptoms, neg class (0) control posts

        rf_classifier_lda = RandomForestClassifier(n_estimators=100, random_state=42)

        cv_lda = KFold(n_splits=5, shuffle=True, random_state=42)
        cv_results_lda = cross_validate(rf_classifier_lda, X=X_lda, y=y_lda, cv=cv_lda, scoring='roc_auc', return_train_score=True)
        # splits the data into training and testing for each fold
        # internally fits the model on training and evaluate on testing

        mean_train_auc_lda = round(cv_results_lda['train_score'].mean(), 3)
        mean_test_auc_lda = round(cv_results_lda['test_score'].mean(), 3)

        print("Training AUC Scores:", cv_results_lda['train_score'])
        print("Testing AUC Scores:", cv_results_lda['test_score'])
        print(f"Mean Training AUC: {mean_train_auc_lda:.3f}")
        print(f"Mean Testing AUC: {mean_test_auc_lda:.3f}")
        print("\n")

        if symptom not in results:
            results[symptom] = {"Symptom": symptom}

        results[symptom].update({
            "LDA Test AUC": mean_test_auc_lda,
            "LDA Ref AUC": reference_values.get(symptom, {}).get("LDA", None)
        })

### DistilRoBERTa Evaluation

In [5]:
CACHE_FILE = os.path.join(FOLDER, 'embeddings_cache.npz')

def save_embeddings_to_cache(file_path, embeddings_dict):
    """Save embeddings to a .npz file."""
    np.savez(file_path, **embeddings_dict)

def load_embeddings_from_cache(file_path):
    """Load embeddings from a .npz file."""
    if os.path.exists(file_path):
        return dict(np.load(file_path, allow_pickle=True))
    return {}

In [6]:
def drb_rf_cross_validation():
    """
    It runs 5-fold cross validation with random forest to evaluate my DistilRoBERTa performance.
    """
    embeddings_cache = load_embeddings_from_cache(CACHE_FILE)
    updated_cache = {}

    for symptom, symptom_data in symptom_datasets.items():
        print(f"Running for symptom: {symptom}")
        # --- DistilRoBERTa Workflow ---

        # generate or load embeddings for depression and control datasets
        if symptom in embeddings_cache:
            symptom_embeddings = embeddings_cache[symptom]
        else:
            symptom_embeddings = get_distilroberta_embeddings(symptom_data['text'].tolist(), tokenizer, model, device)
            updated_cache[symptom] = symptom_embeddings

        if "control" in embeddings_cache:
            control_embeddings = embeddings_cache["control"]
        else:
            control_embeddings = get_distilroberta_embeddings(pre_control['text'].tolist(), tokenizer, model, device)
            updated_cache["control"] = control_embeddings

        # Create labels
        symptom_labels = [1] * len(symptom_data)
        control_labels = [0] * len(pre_control)

        X_roberta = np.vstack([symptom_embeddings, control_embeddings])
        y_roberta = np.array(symptom_labels + control_labels)
        rf_classifier_roberta = RandomForestClassifier(n_estimators=100, random_state=42)
        cv_roberta = KFold(n_splits=5, shuffle=True, random_state=42)
        cv_results_roberta = cross_validate(rf_classifier_roberta, X=X_roberta, y=y_roberta, cv=cv_roberta, scoring='roc_auc', return_train_score=True)

        mean_train_auc_roberta = round(cv_results_roberta['train_score'].mean(), 3)
        mean_test_auc_roberta = round(cv_results_roberta['test_score'].mean(), 3)

        print("DistilRoBERTa Training AUC:", cv_results_roberta['train_score'])
        print("DistilRoBERTa Testing AUC:", cv_results_roberta['test_score'])
        print(f"DistilRoBERTa Mean Training AUC: {mean_train_auc_roberta:.3f}")
        print(f"DistilRoBERTa Mean Testing AUC: {mean_test_auc_roberta:.3f}")
        print("\n")

        if symptom not in results:
            results[symptom] = {"Symptom": symptom}
        results[symptom].update({
            "DRB Test AUC": mean_test_auc_roberta,
            "DRB Ref AUC": reference_values.get(symptom, {}).get("DistilRoBERTa", None)
        })
    save_embeddings_to_cache(CACHE_FILE, {**embeddings_cache, **updated_cache})

### Main function

In [ ]:
results = {}

def main():
    """
    Run 5-fold cross-validation with Random Forest to evaluate both LDA and DistilRoBERTa performance.
    """
    print("\n" + "=" * 40)
    print("Starting LDA 5-Fold Cross-Validation".center(40))
    print("=" * 40 + "\n")
    lda_rf_cross_validation()

    print("\n" + "=" * 40)
    print("Starting DistilRoBERTa 5-Fold Cross-Validation".center(40))
    print("=" * 40 + "\n")
    drb_rf_cross_validation()
main()


  Starting LDA 5-Fold Cross-Validation  

Running for symptom: Anger
Training AUC Scores: [0.99734031 0.99792618 0.99797582 0.99797945 0.99742957]
Testing AUC Scores: [0.95238439 0.91555877 0.90040637 0.92349157 0.92544824]
Mean Training AUC: 0.998
Mean Testing AUC: 0.923


Running for symptom: Anhedonia
Training AUC Scores: [0.99919238 0.99924774 0.99919299 0.9991985  0.99925415]
Testing AUC Scores: [0.96124887 0.95511607 0.95314033 0.95578963 0.95771058]
Mean Training AUC: 0.999
Mean Testing AUC: 0.957


Running for symptom: Anxiety
Training AUC Scores: [0.99929793 0.99928354 0.99934495 0.99947666 0.99945154]
Testing AUC Scores: [0.93942521 0.94029177 0.93678127 0.94036009 0.93101944]
Mean Training AUC: 0.999
Mean Testing AUC: 0.938


Running for symptom: Disordered Eating
Training AUC Scores: [0.99896262 0.99891384 0.99900155 0.99890079 0.99892565]
Testing AUC Scores: [0.97547524 0.97011235 0.9641529  0.95866266 0.96686334]
Mean Training AUC: 0.999
Mean Testing AUC: 0.967


Running

In [ ]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

results_df = pd.DataFrame(list(results.values()))
print("\n--- Final AUC Results with References ---")
print(results_df)


--- Final AUC Results with References ---
              Symptom  LDA Test AUC  LDA Ref AUC  DRB Test AUC  DRB Ref AUC
0               Anger         0.923        0.794         0.944        0.928
1           Anhedonia         0.957        0.906         0.959        0.956
2             Anxiety         0.938        0.837         0.956        0.952
3   Disordered Eating         0.967        0.905         0.955        0.952
4          Loneliness         0.842        0.806         0.916        0.907
5            Sad Mood         0.847        0.788         0.935        0.919
6       Self-Loathing         0.867        0.815         0.934        0.922
7      Sleep Problems         0.977        0.909         0.958        0.956
8  Somatic Complaints         0.914        0.880         0.931        0.925
9       Worthlessness         0.767        0.700         0.924        0.897


### Optional F1 Score Results

In [22]:
# F1 reference values from the table
f1_reference = {
    "Anger": {"LDA": 0.387, "DistilRoBERTa": 0.550},
    "Anhedonia": {"LDA": 0.858, "DistilRoBERTa": 0.906},
    "Anxiety": {"LDA": 0.918, "DistilRoBERTa": 0.959},
    "Disordered Eating": {"LDA": 0.785, "DistilRoBERTa": 0.827},
    "Loneliness": {"LDA": 0.845, "DistilRoBERTa": 0.907},
    "Sad Mood": {"LDA": 0.609, "DistilRoBERTa": 0.755},
    "Self-Loathing": {"LDA": 0.834, "DistilRoBERTa": 0.906},
    "Sleep Problems": {"LDA": 0.826, "DistilRoBERTa": 0.882},
    "Somatic Complaints": {"LDA": 0.855, "DistilRoBERTa": 0.891},
    "Worthlessness": {"LDA": 0.449, "DistilRoBERTa": 0.663},
}

F1_CACHE_FILE = os.path.join(FOLDER, "f1_results_cache.pkl")

def save_f1_cache(data):
    with open(F1_CACHE_FILE, "wb") as f:
        pickle.dump(data, f)

def load_f1_cache():
    if os.path.exists(F1_CACHE_FILE):
        with open(F1_CACHE_FILE, "rb") as f:
            return pickle.load(f)
    return {}

def rf_cross_validation_f1_cached():
    """
    Perform cross-validation for LDA and DistilRoBERTa using F1 scores
    and cache the results after processing each symptom.
    """
    f1_results = load_f1_cache()
    embeddings_cache = load_embeddings_from_cache(CACHE_FILE)

    for symptom, symptom_data in symptom_datasets.items():
        if symptom in f1_results:
            continue

        # --- LDA Workflow ---
        symptom_corpus = [dictionary.doc2bow(doc) for doc in symptom_data['tokens']]
        num_topics = lda_model.num_topics
        X_symptom = get_topic_distributions(symptom_corpus, lda_model, num_topics)
        X_control = get_topic_distributions(control_corpus, lda_model, num_topics)

        X_lda = np.vstack([X_symptom, X_control])
        y_lda = np.hstack([np.ones(X_symptom.shape[0]), np.zeros(X_control.shape[0])])

        rf_classifier_lda = RandomForestClassifier(n_estimators=100, random_state=42)
        cv_lda = KFold(n_splits=5, shuffle=True, random_state=42)
        cv_results_lda = cross_validate(
            rf_classifier_lda, X=X_lda, y=y_lda, cv=cv_lda, scoring='f1', return_train_score=True
        )
        mean_test_f1_lda = round(cv_results_lda['test_score'].mean(), 3)

        # --- DistilRoBERTa Workflow ---
        if symptom in embeddings_cache:
            symptom_embeddings = embeddings_cache[symptom]
        if "control" in embeddings_cache:
            control_embeddings = embeddings_cache["control"]

        symptom_labels = [1] * len(symptom_data)
        control_labels = [0] * len(pre_control)

        X_roberta = np.vstack([symptom_embeddings, control_embeddings])
        y_roberta = np.array(symptom_labels + control_labels)

        rf_classifier_roberta = RandomForestClassifier(n_estimators=100, random_state=42)
        cv_roberta = KFold(n_splits=5, shuffle=True, random_state=42)
        cv_results_roberta = cross_validate(
            rf_classifier_roberta, X=X_roberta, y=y_roberta, cv=cv_roberta, scoring='f1', return_train_score=True
        )
        mean_test_f1_roberta = round(cv_results_roberta['test_score'].mean(), 3)

        f1_results[symptom] = {
            "Symptom": symptom,
            "LDA Test F1": mean_test_f1_lda,
            "LDA Ref F1": f1_reference[symptom]["LDA"],
            "RoBERTa Test F1": mean_test_f1_roberta,
            "RoBERTa Ref F1": f1_reference[symptom]["DistilRoBERTa"],
        }

        save_f1_cache(f1_results)

    f1_results_df = pd.DataFrame(f1_results.values())
    f1_results_df.rename(columns={
        "RoBERTa Test F1": "DRB Test F1",
        "RoBERTa Ref F1": "DRB Ref F1"
    }, inplace=True)

    print("\n--- Final F1 Results with References ---")
    print(f1_results_df)

    return f1_results_df

f1_results_df = rf_cross_validation_f1_cached()


--- Final F1 Results with References ---
              Symptom  LDA Test F1  LDA Ref F1  DRB Test F1  DRB Ref F1
0               Anger        0.678       0.387        0.608       0.550
1           Anhedonia        0.919       0.858        0.907       0.906
2             Anxiety        0.946       0.918        0.964       0.959
3   Disordered Eating        0.902       0.785        0.821       0.827
4          Loneliness        0.872       0.845        0.911       0.907
5            Sad Mood        0.701       0.609        0.779       0.755
6       Self-Loathing        0.862       0.834        0.916       0.906
7      Sleep Problems        0.939       0.826        0.872       0.882
8  Somatic Complaints        0.884       0.855        0.902       0.891
9       Worthlessness        0.497       0.449        0.721       0.663
